### Import Packages

In [16]:
# import packages
import pandas as pd

import re

import nltk
from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
#ps = PorterStemmer()
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag

from nltk.corpus import conll2000

from textblob import TextBlob

from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
# Declaring Snowball Stemmer
snowball = SnowballStemmer("english")

 # declaring WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

### Extract Data

In [18]:
# Extract data
raw_data = pd.read_excel('Email Data.xlsx')
data = raw_data[['Incident','Email_Text']]
data

,Incident,Email_Text
0,5632054,Auto Response Channel: No ChannelNo Channel ...
1,5632083,Response Channel: EmailEmail 01/02/2020 02:2...
2,5632093,Response Channel: EmailEmail 01/21/2020 08:4...
3,5632101,Auto Response Channel: No ChannelNo Channel ...
4,5632108,Auto Response Channel: No ChannelNo Channel ...
...,...,...
822,5670263,Auto Response Channel: No ChannelNo Channel ...
823,5670303,Response Channel: EmailEmail 02/03/2020 05:2...
824,5670353,Auto Response Channel: No ChannelNo Channel ...
825,5670463,Response Channel: EmailEmail 02/05/2020 02:5...


### Cleaning Data

In [19]:
# split email using date wise in the format  01/13/2020 02:20

def split_date(text):
    text = re.split(r' \d?\d/\d?\d/\d{4} [0-1][0-9]:[0-5][0-9]' , text)
    return text

data['Split_Emails']= ''

for i in range(data.shape[0]):
   # print (i)
    data['Split_Emails'][i] = split_date(data['Email_Text'][i])    

C:\Users\II00083764\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\II00083764\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [20]:
data

,Incident,Email_Text,Split_Emails
0,5632054,Auto Response Channel: No ChannelNo Channel ...,[Auto Response Channel: No ChannelNo Channel ...
1,5632083,Response Channel: EmailEmail 01/02/2020 02:2...,"[Response Channel: EmailEmail , AMJames Parv..."
2,5632093,Response Channel: EmailEmail 01/21/2020 08:4...,"[Response Channel: EmailEmail , PMTom Singh,..."
3,5632101,Auto Response Channel: No ChannelNo Channel ...,[Auto Response Channel: No ChannelNo Channel ...
4,5632108,Auto Response Channel: No ChannelNo Channel ...,[Auto Response Channel: No ChannelNo Channel ...
...,...,...,...
822,5670263,Auto Response Channel: No ChannelNo Channel ...,[Auto Response Channel: No ChannelNo Channel ...
823,5670303,Response Channel: EmailEmail 02/03/2020 05:2...,"[Response Channel: EmailEmail , AMJames Parv..."
824,5670353,Auto Response Channel: No ChannelNo Channel ...,[Auto Response Channel: No ChannelNo Channel ...
825,5670463,Response Channel: EmailEmail 02/05/2020 02:5...,"[Response Channel: EmailEmail , AMDerick Upa..."


In [21]:
# padding a list to form the same length and same columns
# It is difficult to split into different columns as the column length is not equal
    
data['NewSplit_Emails'] = ''

for i in range(data.shape[0]):
    
    data['NewSplit_Emails'][i] = data['Split_Emails'][i] + [''] * (42 - len(data['Split_Emails'][i]))
    

C:\Users\II00083764\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\II00083764\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
# checking the length of the column

data['length'] = data['Split_Emails'].apply(lambda x: len(x))

# to extend the split data into different columns
   
data2 = data.iloc[:,[3]]

df2 = pd.DataFrame(data2.NewSplit_Emails.values.tolist(), index= data2.index)

C:\Users\II00083764\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
# df2.to_excel('sp.xlsx')

In [24]:
# to extract the last email from each text

df2['lastemail']=''
for row in range(df2.shape[0]):
    col= data['length'][row]
    col = col-1
    df2['lastemail'][row]=df2.iloc[row,col]


In [25]:
# create the final dataframe by concatenating 2 dataframes

main_data = pd.concat([data, df2], axis=1, sort=False)
# result with all the columns
main_data = main_data.drop(['length', 'NewSplit_Emails','Split_Emails'],axis = 1)

In [26]:
# this is the required result with last column
#result dataframe with only the last column
result = main_data[['Incident','Email_Text','lastemail']]

In [27]:
# removing Best regards and everything before Dear word in each cell

def data_cleaning(text):
    try:
        text = re.sub('\n+',' ',text)
        text = re.sub(r' Best regards.*$','',text)
        text = re.sub(r' Sincerely.*$','',text)
        text = re.sub(r'.*Dear ','',text)
        text = re.sub(r'.*.pdf','',text)
        text = re.sub(r'.*.JPG','',text) 
        text = re.sub(r'.*.jpg','',text)
        text = re.sub(r'.*.jpeg','',text)
        text = re.sub(r'.*.png','',text)
        text = re.sub(r'.*.bmp','',text)
        text = re.sub(r'.*.gif','',text)
        text = re.sub(r'.*.docx','',text)
        text = re.sub(r'.*.PNG','',text)
        text = text.strip()
    except:
        return text

    return text


for col in result:
    #if col=='Email_Text' or col== 'Incident':
    #    continue
    for i in range(result.shape[0]):
        #print(result[col][i])
        text = result[col][i]
        result[col][i] = data_cleaning(text)
        

C:\Users\II00083764\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
#result.to_excel('outputfile.xlsx')

###### NTLK - NER  to remove names from text

In [17]:
"""
def preprocess_text(text):
    
    This function takes a text. Split it in tokens using word_tokenize.  
    And then tags them using pos_tag from NLTK module. 
    It outputs a list of tuples. Each tuple consists of a word and the tag with its  
    part of speech. 
    
    # Get the tokens 
    tokens = nltk.word_tokenize(text) 
    # Tags the tokens 
    tagging = nltk.pos_tag(tokens) 
    # Returns the list of tuples 
    return tagging 
    
"""

'\ndef preprocess_text(text):\n    \n    This function takes a text. Split it in tokens using word_tokenize.  \n    And then tags them using pos_tag from NLTK module. \n    It outputs a list of tuples. Each tuple consists of a word and the tag with its  \n    part of speech. \n    \n    # Get the tokens \n    tokens = nltk.word_tokenize(text) \n    # Tags the tokens \n    tagging = nltk.pos_tag(tokens) \n    # Returns the list of tuples \n    return tagging \n    \n'

In [18]:
"""
def Named_entity_recognition(text):                
    # Split and label the text 
    label_text = preprocess_text(text) 
    #print
    entity1 = []
    #Print first 20 tuples, 5 per line 
    for i in range(0, 20, 5): 
        print(label_text[i], label_text[i+1], label_text[i+2], label_text[i+3],  
        label_text[i+4])
        
    # Define the rule
    rule = "NP: {<DT>?<JJ>*<NNP>+}" 
    
    # We define the parser using the rule 
    parser = nltk.RegexpParser(rule) 

    # Apply to the tagged words 
    result = parser.parse(label_text)
    #print (result)
      
    # Print only the chunks
    entities = nltk.ne_chunk(label_text,binary = False)
    #for entity in entities:
    for entity in entities:
        print(entity)
        if type(entity) == nltk.tree.Tree:
            #if entity.label_ == PERSON:
            print(entity)
            entity1.append(entity)
     
    return entity1
"""

'\ndef Named_entity_recognition(text):                \n    # Split and label the text \n    label_text = preprocess_text(text) \n    #print\n    entity1 = []\n    #Print first 20 tuples, 5 per line \n    for i in range(0, 20, 5): \n        print(label_text[i], label_text[i+1], label_text[i+2], label_text[i+3],  \n        label_text[i+4])\n        \n    # Define the rule\n    rule = "NP: {<DT>?<JJ>*<NNP>+}" \n    \n    # We define the parser using the rule \n    parser = nltk.RegexpParser(rule) \n\n    # Apply to the tagged words \n    result = parser.parse(label_text)\n    #print (result)\n      \n    # Print only the chunks\n    entities = nltk.ne_chunk(label_text,binary = False)\n    #for entity in entities:\n    for entity in entities:\n        print(entity)\n        if type(entity) == nltk.tree.Tree:\n            #if entity.label_ == PERSON:\n            print(entity)\n            entity1.append(entity)\n     \n    return entity1\n'

In [19]:
"""
# concatenating all the text required into a long string to find the names of Persons in the string
strings = ''
for col in result:
    if col=='Email_Text' or col== 'Incident' or col=='0':
        continue
    for i in range(result.shape[0]):
        # print(result[col][i])
        text = result[col][i]
        #text = text.strip().split() 
        if len(text.strip().split())< 10:
            continue
        else:
            text1 = result[col][i]
            text1 = text1.strip()
            text = text + text1
    
    strings = strings + text
   # print(string)
    
Entity_recog = Named_entity_recognition(strings)
#print (Entity_recog)

"""


"\n# concatenating all the text required into a long string to find the names of Persons in the string\nstrings = ''\nfor col in result:\n    if col=='Email_Text' or col== 'Incident' or col=='0':\n        continue\n    for i in range(result.shape[0]):\n        # print(result[col][i])\n        text = result[col][i]\n        #text = text.strip().split() \n        if len(text.strip().split())< 10:\n            continue\n        else:\n            text1 = result[col][i]\n            text1 = text1.strip()\n            text = text + text1\n    \n    strings = strings + text\n   # print(string)\n    \nEntity_recog = Named_entity_recognition(strings)\n#print (Entity_recog)\n\n"

In [67]:
# testing this code

strings = ''
for col in result:
    if col=='Email_Text' or col== 'Incident' or col=='0':
        continue
    for i in range(result.shape[0]):
        # print(result[col][i])
        text = result[col][i]
        #text = text.strip().split() 
        if len(text.strip().split())< 10:
            continue
        else:
            text1 = result[col][i]
            tagged_sentence = nltk.tag.pos_tag(text1.split())
            edited_sentence = [word for word,tag in tagged_sentence if tag != 'NNP' and tag != 'NNPS']
            result[col][i] = (' '.join(edited_sentence))            


C:\Users\II00083764\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


###### Text Blob  - please skip

In [46]:
"""

# concatenating all the text required into a long string to find the names of Persons in the string
strings = []

for col in result:
    if col=='Email_Text' or col== 'Incident' or col=='0':
        continue
    for i in range(result.shape[0]):
        
        text = result[col][i]
        Txtblob = TextBlob(text)
    
        for noun_phrase in Txtblob.noun_phrases:
            strings.append(noun_phrase)
        
strings= list(set(strings))
print (strings)

for col in result:
    if col=='Email_Text' or col== 'Incident' or col=='0':
        continue
    for i in range(result.shape[0]):
        
        text = result[col][i]
        Txtblob = TextBlob(text)
        
        edited_sentence = ' '.join([word for word in text.split(' ') if word not in strings ])
        result[col][i] = edited_sentence
        

for col in result:
    if col=='Email_Text' or col== 'Incident' or col=='0':
        continue
    for i in range(result.shape[0]):
        text = result[col][i]        
        edited_sentence = ' '.join([word for word in text.split(' ') if word not in strings ])
        result[col][i] = edited_sentence

    #for word , pos in enumerate(Txtblob.tags):
    #    print (word,pos)
    #    if tag != '*,NNP':
    #        print(word)
    #        print(''.join(word))
    
"""

"\n\n# concatenating all the text required into a long string to find the names of Persons in the string\nstrings = []\n\nfor col in result:\n    if col=='Email_Text' or col== 'Incident' or col=='0':\n        continue\n    for i in range(result.shape[0]):\n        \n        text = result[col][i]\n        Txtblob = TextBlob(text)\n    \n        for noun_phrase in Txtblob.noun_phrases:\n            strings.append(noun_phrase)\n        \nstrings= list(set(strings))\nprint (strings)\n\nfor col in result:\n    if col=='Email_Text' or col== 'Incident' or col=='0':\n        continue\n    for i in range(result.shape[0]):\n        \n        text = result[col][i]\n        Txtblob = TextBlob(text)\n        \n        edited_sentence = ' '.join([word for word in text.split(' ') if word not in strings ])\n        result[col][i] = edited_sentence\n        \n\nfor col in result:\n    if col=='Email_Text' or col== 'Incident' or col=='0':\n        continue\n    for i in range(result.shape[0]):\n        

In [47]:
result.to_excel('outputfile.xlsx')

FileCreateError: [Errno 13] Permission denied: 'outputfile.xlsx'

### Stemming and lemmatization

In [29]:

"""  
    text = re.sub('\[.*?\]', '', text)
    
    text = re.sub('<.*?>+', '', text)
    # text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    text = re.sub(r'\r', "", text)
    text = re.sub(r"(@.+$|\\..{1,3}$)",'',text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~.!?,%'']", "", text)
    text = re.sub(r'http:\/\/.*', '', text)
    text = re.sub(r"[0-9]", "", text)    
    text= word_tokenize(text)
    # stemming and removing stop words
    text = [snowball.stem(word) for word in text if not word in set(stopwords.words('english'))]
    # Lemmatization
    text = [lemmatizer.lemmatize(token) for token in text]
    text = ' '.join(text)
"""
def stemming_lemmatization(text):
    text = text.lower()
    
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'http:\/\/.*', '', text)
    text = re.sub(r"\(?(\d{3})\)?[ .-]?(\d{3})[ .-]?(\d{4})", '', text)
    
    return text

result['edited_email']=''

for col in result:
    if col=='Email_Text' or col== 'Incident':
        continue
    for i in range(result.shape[0]):
        text = result[col][i]
        result['edited_email'][i] = stemming_lemmatization(text)

C:\Users\II00083764\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\II00083764\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


##### Number of words in a paragraph

In [30]:
result.to_excel("Outputfile2.xlsx")

In [ ]:
# find the number of words
text =''

for col in result:
    if col=='Email_Text' or col== 'Incident':
        continue
    for i in range(result.shape[0]):
        text = text + result[col][i]        

words = text.split(' ')

In [ ]:
# to find unique words
words = list(set(words))

#### Vectorizer

In [15]:
tf = TfidfVectorizer()

txt1 = result['lastemail']

txt_fitted = tf.fit_transform(txt1)
txt_fitted = txt_fitted.toarray()

txt_fitted

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])